In [11]:
from src.routing import create_data, get_options_df, solve, prelim_check
import time
import pandas as pd
import numpy as np

In [12]:
def save_results(results_df, filename='vrp_results.csv'):
    results_df.to_csv(filename, index=False)
    print(f"Results saved to {filename}")
    
def load_results(filename='vrp_results.csv'):
    import os
    if os.path.exists(filename):
        return pd.read_csv(filename)
    else:
        print(f"No existing file found. A new DataFrame will be initialized.")
        return pd.DataFrame()  # Return an empty DataFrame if the file does not exist

def append_and_save_new_results(new_results, filename='vrp_results.csv'):
    existing_results_df = load_results(filename)
    # Concatenate new results to the existing DataFrame
    updated_results_df = pd.concat([existing_results_df, pd.DataFrame(new_results)], ignore_index=True)
    # Save the updated DataFrame
    save_results(updated_results_df, filename)

In [13]:
# Work_schedule
work_schedule = {
    1: [7 * 60, 19 * 60],
    2: [18 * 60, 18 * 60],
    3: [18 * 60, 18 * 60],
    4: [19 * 60, 19 * 60],
    5: [17 * 60, 17 * 60],
    6: [18 * 60, 18 * 60],
    7: [18 * 60, 18 * 60]
}

In [14]:
num_nodes = 100
min_work_days = 5
percentage_of_appointments = 0.2

overnight_stays_ok = {1}
min_distance_overnight = 50
lunch_earliest_start = 12
lunch_duration = 30
fix_app_margin = 10

day_lengths = {day: (end - start) for day, (start, end) in work_schedule.items()}
days_off = {day for day, length in day_lengths.items() if length == 0}
work_days = {day for day in day_lengths if day not in days_off}
max_day_length = max(day_lengths.values())
relative_day_lengths = [round(length / max_day_length, 2) for _, length in day_lengths.items()]
no_overnight_stays = {day for day in day_lengths if day not in days_off and day not in overnight_stays_ok}
max_days_off = len(days_off)
n_work_days = 7 - max_days_off

# Algorithm
#penalty_factor = [10, 100000, 200] # exponent (more spread between fixed/nonfixed) & max/min prio
overnight_cost = 2
GlobalSpanCostCoefficient = 0
slack = 200000

nodes_df, time_matrix = create_data(num_nodes, percentage_of_appointments, home_node_id=0, min_work_days=min_work_days, days_off=days_off, visiting_interval_min=10, visiting_interval_max=30, max_last_visit=30, simple_schedule=False)
nodes_df, messages = prelim_check(nodes_df, time_matrix, work_schedule, fix_app_margin, work_days, verbose=False)

In [15]:
total_hours = 0
for day, hours in work_schedule.items():
    if not day in days_off:
        total_hours += hours[1] - hours[0]

n_fixed_appointments = sum(nodes_df['fixed_appointment'].notnull())

avg_spread = time_matrix.iloc[nodes_df['node_id'].values, nodes_df['node_id'].values].median().median()
max_spread = time_matrix.iloc[nodes_df['node_id'].values, nodes_df['node_id'].values].max().max()
options_df = get_options_df(days_off, relative_day_lengths, no_overnight_stays, max_days_off)
time_limit = 10
num_nodes_to_consider = 8

nodes_df = nodes_df.sort_values('priority', ascending=False, inplace=False)
top_nodes_df = nodes_df.head(num_nodes_to_consider)
if 0 not in top_nodes_df['node_id'].values:
    node_zero_df = nodes_df[nodes_df['node_id'] == 0]
    nodes_df = pd.concat([node_zero_df, top_nodes_df]).head(num_nodes_to_consider)
else:
    nodes_df = top_nodes_df
nodes_df = nodes_df.sort_values('node_id', ascending=True, inplace=False)

existing_results_df = load_results()
max_index = existing_results_df['index'].max() if len(existing_results_df) > 0 else 0

results = []

first_algorithm = 'SAVINGS' # 'PATH_CHEAPEST_ARC', 'SAVINGS', 'PARALLEL_CHEAPEST_INSERTION', 'BEST_INSERTION'
second_algorithm = 'GUIDED_LOCAL_SEARCH' # 'GUIDED_LOCAL_SEARCH', 'GREEDY_DESCENT'

start_time = time.time()

In [21]:
start_node = 55
updated_options_df, figs, routes, updated_nodes_df = solve(options_df, nodes_df, time_matrix, slack, work_schedule, fix_app_margin, time_limit, lunch_duration, first_algorithm, second_algorithm, GlobalSpanCostCoefficient, min_distance_overnight, start_node, verbose=False, visual=True, restrictive=False)

# Stop timing
end_time = time.time()
duration = end_time - start_time

# Process and extract relevant details
max_app_cons = updated_options_df['fixed_app_cons'].max()
best_solutions = updated_options_df[updated_options_df['fixed_app_cons'] == max_app_cons]
best_solutions['num_nodes_cons'] -= best_solutions['n_overnight_trips'] * overnight_cost
best_solution = best_solutions['num_nodes_cons'].idxmax()
best_option_df = updated_options_df.iloc[best_solution]

0     {1}
1     {1}
36    {1}
44    {1}
47    {1}
55    {1}
68    {1}
73    {1}
Name: cluster, dtype: object
end index: 4 internal start index: 5, data depot: 0
location index: 0 and initial index 0, windows: [[420, 1140]]
location index: 1 and initial index 1, windows: [[480, 892]]
location index: 2 and initial index 36, windows: [[1032, 1032]]
location index: 3 and initial index 44, windows: [[480, 962]]
location index: 4 and initial index 47, windows: [[775, 775]]
location index: 5 and initial index 55, windows: [[480, 908]]
location index: 6 and initial index 68, windows: [[480, 931]]
location index: 7 and initial index 73, windows: [[661, 661]]
Option 0 with objective value 510 and 0 dropped nodes


In [18]:
# # Collect results
# result = {
#     "index": max_index + 1,
#     "time": pd.Timestamp.now().strftime('%Y-%m-%d %H:%M'),
#     "in_first_algo": first_algorithm,
#     "in_second_algo": second_algorithm,
#     "in_penalty_exp": np.nan,
#     "in_penalty_max": np.nan,
#     "in_penalty_min": np.nan,
#     "in_fixed_app_fraction": percentage_of_appointments,
    
#     "thr_num_nodes_to_consider": num_nodes_to_consider,
#     "thr_num_fixed_appointments": n_fixed_appointments,
#     "thr_n_overnight_trips": best_option_df['n_overnight_trips'],
#     "thr_total_hours": total_hours,
#     "thr_avg_spread": round(avg_spread, 2),
#     "thr_max_spread": round(max_spread, 2),
#     "thr_n_work_days": n_work_days,
#     "thr_time_limit": time_limit,

#     "out_obj_value": best_option_df["obj_value"],
#     "out_num_nodes_cons": best_option_df['num_nodes_cons'],
#     "out_rel_nodes_cons": round(best_option_df['nodes_cons'],2),
#     "out_rel_fixed_app_cons": best_option_df['fixed_app_cons'] if len(updated_options_df['fixed_appointments_nodes'].values[0]) >= 1 else np.nan,
#     "out_time_taken": round(duration, 2)
# }
# results.append(result)

# # Convert list of dictionaries to DataFrame
# new_results_df = pd.DataFrame(results)

# append_and_save_new_results(new_results_df)